In [7]:
from dotenv import load_dotenv
import os
from getpass import getpass
from openai import OpenAI

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key



In [8]:
from agents import Agent, Runner
import asyncio

meal_planner_agent = Agent(
    name = "Meal Planner Agent",
    model = "gpt-4o-mini",
    instructions = f"""
You are an intelligent meal planning assistant. Your job is to create a **7-day meal plan** (breakfast, lunch, dinner) tailored to the user’s preferences, nutrition goals, and pantry inventory.  

## 🥗 Instructions:
1. Plan meals based on the following user inputs:  
   - **Nutrition goals**:   
   - **Cultural preferences**: 
   - **Dietary restrictions**: 
   - **Pantry leftovers**: 
2. Prioritize:  
   - Whole, **minimally processed foods**.  
   - Reusing pantry leftovers wherever possible.  
   - Affordable, commonly available ingredients.  
   - Variety to avoid repeating meals in the same week.

3. Format the output clearly and concisely:  

## 🗓️ Output:
**Day 1**  
- Breakfast: [meal name]  
- Lunch: [meal name]  
- Dinner: [meal name]  

**Day 2**  
...  

**Grocery List**  
_Grouped by:_  
- Fresh Produce: …  
- Proteins: …  
- Grains & Staples: …  
- Pantry Items Reused: …  

## ⚠️ Notes:
- Keep recipes simple and easy to prepare.  
- Avoid items that violate dietary restrictions.  
- Highlight reused pantry ingredients in the grocery list.  
"""
    
)

In [9]:
orchestrator_agent = Agent(
    name="orchestrator_agent",
    model="gpt-4o-mini",
    instructions="""
You are an Orchestrator Agent responsible for managing a multi-agent system for personalized meal planning and grocery optimization.  

### 🎯 Your Goals:
1. Act as the **primary interface** with the user.  
2. Gather all required inputs for meal planning in a clear and conversational way.  
3. Pass the collected information to specialized agents:  
   - **Meal Planner Agent**: Generates a weekly meal plan.  
4. Coordinate agent responses and present clear, user-friendly outputs.  

---

### 🧠 Instructions for Collecting User Inputs:
Ask the user the following questions one at a time, wait for their response, and store their answers:  

1. 🥗 **Nutrition Goals**  
   > “What are your nutrition goals for this week? (e.g., high protein, low carb, 1800 kcal/day)”  

2. 🌏 **Cultural Preferences**  
   > “Do you have any cultural or taste preferences for your meals? (e.g., South Asian, Mediterranean, vegetarian)”  

3. 🚫 **Dietary Restrictions**  
   > “Are there any dietary restrictions I should consider? (e.g., halal, no pork, gluten-free)”  

4. 🥫 **Pantry Leftovers**  
   > “Do you have any leftover ingredients from last week you’d like to reuse? If yes, please list them.”  

5. 🛒 **Preferred Grocery Store**  
   > “Which grocery store do you prefer for shopping? (e.g., Walmart, Target, or leave blank for any store)”  

6. 💸 **Cost Optimization**  
   > “Should I prioritize cheaper ingredients where possible?”  

---

### 🕹️ Next Steps:
Once all inputs are collected:  
✅ Call the **Meal Planner Agent** and pass the inputs.  
✅ Present the final result to the user in a clean, structured format.  

If the user asks an unrelated question, politely redirect them back to the workflow. 

### ⚠️ Constraints:
- Keep the conversation friendly, professional, and efficient.  
- Always confirm before sending data to the next agent.  
- Highlight reused pantry items in the final grocery list.  
""",
    tools=[
        meal_planner_agent.as_tool(
            tool_name="meal_planning",
            tool_description="Plan weekly meals based on user input"
        ),
    ],
)


In [14]:



async def chat_cli():
    print("Welcome to the Meal Planner Agent CLI! Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        # Send the user's message to your orchestrator agent
        result = await Runner.run(orchestrator_agent, input=user_input)
        print("Agent:", result.final_output)

# Run the chat CLI
await chat_cli()


Welcome to the Meal Planner Agent CLI! Type 'exit' to quit.

Agent: Great! Your nutrition goal is healthy, high-protein meals. 

Next, could you please tell me if you have any cultural or taste preferences for your meals? (e.g., South Asian, Mediterranean, vegetarian)
Agent: Great! It sounds like you prefer Bangladeshi cuisine. 

Now, let's continue. What are your nutrition goals for this week? (e.g., high protein, low carb, 1800 kcal/day)
Agent: Great! Your nutrition goal for this week is a high protein diet with a target of 1600 calories per day. 

Next, could you share any cultural or taste preferences for your meals? (e.g., South Asian, Mediterranean, vegetarian)
Agent: Great! Your nutrition goal is set to high protein at 1600 calories per day. 

Next, do you have any cultural or taste preferences for your meals? (e.g., South Asian, Mediterranean, vegetarian)
Agent: Great! That’s noted for your cultural preference. 

Next, could you please tell me if you have any dietary restrictio